In [1]:
import os 
extracted_files = os.listdir('adult')


In [2]:
extracted_files

['adult.test', 'old.adult.names', 'adult.data', 'Index', 'adult.names']

In [3]:
with open('adult/adult.names', 'r') as f:
    names_content = f.read()

print(names_content[:1000])

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

In [5]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder \
    .appName("UCI Adult Income Analysis") \
    .getOrCreate()

# Show the Spark session details to confirm it's up and running
print(spark.version)


3.5.0


In [13]:
from pyspark.sql import SparkSession

# Initialize a Spark session if you haven't already
spark = SparkSession.builder \
    .appName("UCI Adult Income Analysis") \
    .getOrCreate()

# Define the schema based on what we know from the adult.names file
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", IntegerType(), True),
    StructField("education", StringType(), True),
    StructField("education_num", IntegerType(), True),
    StructField("marital_status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital_gain", IntegerType(), True),
    StructField("capital_loss", IntegerType(), True),
    StructField("hours_per_week", IntegerType(), True),
    StructField("native_country", StringType(), True),
    StructField("income", StringType(), True)
])

# Load the data into a DataFrame
data_path = "adult/adult.data"
df = spark.read.csv(data_path, schema=schema, header=False, sep=",")

# Show the first few rows to verify
df.__sizeof__()


32

In [19]:
import  pandas as pd 
df = spark.read.csv("adult/adult.data")

In [24]:
df.show()

+---+-----------------+-------+-------------+---+--------------------+------------------+--------------+-------------------+-------+------+----+----+--------------+------+
|_c0|              _c1|    _c2|          _c3|_c4|                 _c5|               _c6|           _c7|                _c8|    _c9|  _c10|_c11|_c12|          _c13|  _c14|
+---+-----------------+-------+-------------+---+--------------------+------------------+--------------+-------------------+-------+------+----+----+--------------+------+
| 39|        State-gov|  77516|    Bachelors| 13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|  2174|   0|  40| United-States| <=50K|
| 50| Self-emp-not-inc|  83311|    Bachelors| 13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|     0|   0|  13| United-States| <=50K|
| 38|          Private| 215646|      HS-grad|  9|            Divorced| Handlers-cleaners| Not-in-family|              White|   Male|     0| 